# 📊 Exploration du Dataset MNIST

Ce notebook vous permet de découvrir le dataset MNIST, qui est l'un des datasets les plus célèbres en machine learning.

## Qu'est-ce que MNIST ?

**MNIST** (Modified National Institute of Standards and Technology) est un dataset de **70,000 images** de chiffres manuscrits (0-9) :

- **60,000 images** pour l'entraînement
- **10,000 images** pour le test
- Chaque image est en **niveaux de gris** de taille **28×28 pixels**
- Créé par Yann LeCun et al. dans les années 1990

C'est le "Hello World" du deep learning !

## 1. Import des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Ajouter le répertoire parent au path pour importer nos modules
sys.path.append('..')

from src.utils import load_mnist, get_data_stats

# Configuration pour de beaux graphiques
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✓ Bibliothèques importées avec succès")

## 2. Téléchargement et chargement des données

La première fois que vous exécutez cette cellule, le dataset sera téléchargé automatiquement (~11 MB).

In [ ]:
# Charger MNIST (téléchargement automatique si nécessaire)
X_train, y_train, X_test, y_test = load_mnist(
    data_dir='../data',
    flatten=False,  # Garder le format (28, 28)
    normalize=True  # Normaliser les pixels [0, 255] -> [0, 1]
)

print("\n✓ Données chargées avec succès !")
print(f"\nForme des données:")
print(f"  X_train: {X_train.shape}  (60000 images de 28x28)")
print(f"  y_train: {y_train.shape}  (60000 labels)")
print(f"  X_test:  {X_test.shape}   (10000 images de 28x28)")
print(f"  y_test:  {y_test.shape}   (10000 labels)")

## 3. Statistiques du dataset

In [ ]:
# Afficher les statistiques
print("DATASET D'ENTRAÎNEMENT")
get_data_stats(X_train, y_train)

print("\nDATASET DE TEST")
get_data_stats(X_test, y_test)

## 4. Visualisation d'exemples

Affichons quelques images du dataset pour voir à quoi elles ressemblent.

In [ ]:
def plot_images(images, labels, num_images=25, title="Exemples MNIST"):
    """
    Affiche une grille d'images MNIST
    
    Args:
        images: Array d'images (n, 28, 28)
        labels: Array de labels (n,)
        num_images: Nombre d'images à afficher
        title: Titre du graphique
    """
    grid_size = int(np.sqrt(num_images))
    
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(12, 12))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            # Afficher l'image
            ax.imshow(images[i], cmap='gray')
            ax.set_title(f'Label: {labels[i]}', fontsize=12)
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Afficher 25 exemples aléatoires
random_indices = np.random.choice(len(X_train), 25, replace=False)
plot_images(X_train[random_indices], y_train[random_indices], num_images=25)

## 5. Comprendre la structure d'une image

Chaque image est une matrice 28×28 avec des valeurs entre 0 (noir) et 1 (blanc).

In [ ]:
# Prendre une image exemple
sample_idx = 0
sample_image = X_train[sample_idx]
sample_label = y_train[sample_idx]

print(f"Image d'exemple (index {sample_idx}):")
print(f"Label: {sample_label}")
print(f"Forme: {sample_image.shape}")
print(f"Type: {sample_image.dtype}")
print(f"Valeurs min/max: {sample_image.min():.3f} / {sample_image.max():.3f}")

# Affichage côte à côte: image et valeurs numériques
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Image
axes[0].imshow(sample_image, cmap='gray')
axes[0].set_title(f'Image du chiffre {sample_label}', fontsize=14)
axes[0].axis('off')

# Heatmap des valeurs
im = axes[1].imshow(sample_image, cmap='viridis')
axes[1].set_title('Valeurs numériques des pixels', fontsize=14)
axes[1].set_xlabel('Colonne')
axes[1].set_ylabel('Ligne')
plt.colorbar(im, ax=axes[1], label='Intensité')

plt.tight_layout()
plt.show()

# Afficher un extrait des valeurs numériques
print("\nExtrait de la matrice (lignes 10-15, colonnes 10-15):")
print(sample_image[10:15, 10:15])

## 6. Distribution des classes

Vérifions que le dataset est bien équilibré (toutes les classes sont représentées équitablement).

In [ ]:
# Compter le nombre d'occurrences de chaque chiffre
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

# Créer un graphique
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Dataset d'entraînement
axes[0].bar(unique_train, counts_train, color='skyblue', edgecolor='navy')
axes[0].set_xlabel('Chiffre', fontsize=12)
axes[0].set_ylabel('Nombre d\'images', fontsize=12)
axes[0].set_title('Distribution - Entraînement', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(10))
axes[0].grid(axis='y', alpha=0.3)

# Dataset de test
axes[1].bar(unique_test, counts_test, color='lightcoral', edgecolor='darkred')
axes[1].set_xlabel('Chiffre', fontsize=12)
axes[1].set_ylabel('Nombre d\'images', fontsize=12)
axes[1].set_title('Distribution - Test', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(10))
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Le dataset est relativement équilibré !")

## 7. Variabilité des images par classe

Regardons plusieurs exemples de chaque chiffre pour voir la diversité des écritures manuscrites.

In [ ]:
def plot_digit_samples(images, labels, digit, num_samples=10):
    """
    Affiche plusieurs exemples d'un chiffre spécifique
    
    Args:
        images: Array d'images
        labels: Array de labels
        digit: Le chiffre à afficher (0-9)
        num_samples: Nombre d'exemples à afficher
    """
    # Trouver les indices où label == digit
    indices = np.where(labels == digit)[0]
    
    # Sélectionner aléatoirement num_samples indices
    selected_indices = np.random.choice(indices, num_samples, replace=False)
    
    # Afficher
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 2))
    fig.suptitle(f'Exemples du chiffre {digit}', fontsize=14, fontweight='bold')
    
    for i, ax in enumerate(axes):
        ax.imshow(images[selected_indices[i]], cmap='gray')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Afficher des exemples pour chaque chiffre
for digit in range(10):
    plot_digit_samples(X_train, y_train, digit, num_samples=10)

## 8. Analyse des pixels

Calculons la moyenne de tous les pixels pour chaque classe. Cela nous montre les zones les plus fréquemment "actives".

In [ ]:
# Calculer l'image moyenne pour chaque chiffre
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Image moyenne par classe', fontsize=16, fontweight='bold')

for digit in range(10):
    # Trouver toutes les images du chiffre
    digit_images = X_train[y_train == digit]
    
    # Calculer la moyenne
    mean_image = np.mean(digit_images, axis=0)
    
    # Afficher
    row = digit // 5
    col = digit % 5
    axes[row, col].imshow(mean_image, cmap='hot')
    axes[row, col].set_title(f'Chiffre {digit}', fontsize=12)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

## 9. Format pour le réseau de neurones

Pour entraîner un réseau de neurones, nous devons **aplatir** les images 28×28 en vecteurs de 784 valeurs.

In [ ]:
# Charger MNIST avec flatten=True
X_train_flat, y_train_flat, X_test_flat, y_test_flat = load_mnist(
    data_dir='../data',
    flatten=True,    # Aplatir les images
    normalize=True
)

print("Format aplati pour le réseau de neurones:")
print(f"  X_train: {X_train_flat.shape}  (60000 vecteurs de 784 valeurs)")
print(f"  X_test:  {X_test_flat.shape}   (10000 vecteurs de 784 valeurs)")

# Vérification: une image aplatie
sample_flat = X_train_flat[0]
sample_original = X_train[0]

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Image originale
axes[0].imshow(sample_original, cmap='gray')
axes[0].set_title(f'Image originale (28×28)\nLabel: {y_train[0]}', fontsize=12)
axes[0].axis('off')

# Image aplatie (affichée comme un vecteur)
axes[1].plot(sample_flat, linewidth=0.5)
axes[1].set_title(f'Image aplatie (vecteur de 784 valeurs)\nLabel: {y_train_flat[0]}', fontsize=12)
axes[1].set_xlabel('Index du pixel')
axes[1].set_ylabel('Intensité')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nC'est ce format que nous utiliserons pour entraîner notre réseau de neurones !")

## 10. Récapitulatif

### Ce que nous avons appris :

1. **MNIST** contient 70,000 images de chiffres manuscrits (60k train, 10k test)
2. Chaque image est **28×28 pixels** en niveaux de gris
3. Les pixels sont normalisés entre **0 et 1**
4. Le dataset est **équilibré** (chaque chiffre est bien représenté)
5. Il y a une **grande variabilité** dans les écritures manuscrites
6. Pour le réseau de neurones, nous aplatissons les images en **vecteurs de 784 valeurs**

### Prochaine étape :

Dans le prochain notebook (`02_simple_network.ipynb`), nous allons **construire notre premier réseau de neurones** pour classifier ces chiffres !

## 💡 Exercices bonus

Si vous voulez explorer davantage :

1. Trouvez les images les plus "difficiles" (pixels les plus proches de 0.5)
2. Calculez l'écart-type de chaque pixel pour voir où il y a le plus de variabilité
3. Créez une fonction pour afficher les images les plus "typiques" et les plus "atypiques" de chaque classe
4. Analysez la corrélation entre les pixels centraux et les pixels de bord